In [37]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_ = pd.read_csv("/kaggle/input/omnichannel-dataset/flo_data_20k.csv")
df = df_.copy()
df.head()

In [38]:
# define the outlier threshold function
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

# define the function to replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit, 0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)


In [39]:
# handle outliers in the specified columns
columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
           "customer_value_total_ever_online"]
for col in columns:
    replace_with_thresholds(df, col)

In [40]:
# omnichannel customers shop from both online and offline platforms. Create new variables for each customer's total number of purchases and total spending
df = df.assign(
    order_num_total=df['order_num_total_ever_online'] + df['order_num_total_ever_offline'],
    customer_value_total=df['customer_value_total_ever_online'] + df['customer_value_total_ever_offline'])

In [41]:
# convert date variables to datetime type
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [44]:
# CLTV Dataframe
today_date = dt.datetime(2021, 6, 1)
cltv_df = pd.DataFrame()
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
cltv_df["T_weekly"] = (today_date - df["first_order_date"]).dt.days / 7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]
cltv_df = cltv_df[(cltv_df['frequency'] > 0) & (cltv_df['recency_cltv_weekly'] > 0) & (cltv_df['T_weekly'] > 0)]

In [46]:
# BD/NBD Model
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                                                       cltv_df['frequency'],
                                                                                       cltv_df['recency_cltv_weekly'],
                                                                                       cltv_df['T_weekly'])

cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6,
                                                                                       cltv_df['frequency'],
                                                                                       cltv_df['recency_cltv_weekly'],
                                                                                       cltv_df['T_weekly'])

In [47]:
# expected sales in 3 months
cltv_df.sort_values("exp_sales_3_month", ascending=False)[:10]

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,62.71,67.29,52.00,166.22,4.65,9.31
15611,39.71,40.00,29.00,165.30,3.37,6.74
8328,28.86,33.29,25.00,97.44,3.14,6.28
19538,52.57,58.71,31.00,228.53,3.08,6.16
14373,38.00,46.43,27.00,141.35,3.00,6.00
10489,103.14,111.86,43.00,157.11,2.98,5.95
4315,133.14,147.14,49.00,161.85,2.83,5.66
6756,62.71,64.14,29.00,168.88,2.79,5.58
6666,9.71,13.00,17.00,259.87,2.78,5.56
10536,104.57,113.43,40.00,176.20,2.76,5.52


In [49]:
# expected sales in 6 months
cltv_df.sort_values("exp_sales_6_month", ascending=False)[:10]

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,62.71,67.29,52.00,166.22,4.65,9.31
15611,39.71,40.00,29.00,165.30,3.37,6.74
8328,28.86,33.29,25.00,97.44,3.14,6.28
19538,52.57,58.71,31.00,228.53,3.08,6.16
14373,38.00,46.43,27.00,141.35,3.00,6.00
10489,103.14,111.86,43.00,157.11,2.98,5.95
4315,133.14,147.14,49.00,161.85,2.83,5.66
6756,62.71,64.14,29.00,168.88,2.79,5.58
6666,9.71,13.00,17.00,259.87,2.78,5.56
10536,104.57,113.43,40.00,176.20,2.76,5.52


In [51]:
# Gamma-Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                       cltv_df['monetary_cltv_avg'])
cltv_df.head()

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,17.00,30.57,5.00,187.87,0.97,1.95,193.63
1,209.86,224.86,21.00,95.88,0.98,1.97,96.66
2,52.29,78.86,5.00,117.06,0.67,1.34,120.97
3,1.57,20.86,2.00,60.98,0.70,1.40,67.32
4,83.14,95.43,2.00,104.99,0.40,0.79,114.32


In [53]:
# CLTV Prediction
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv
cltv_df.sort_values("cltv", ascending=False)[:20]

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,2.86,7.86,4.00,1401.80,1.09,2.19,1449.05,3324.87
13880,6.14,13.14,11.00,758.09,1.97,3.94,767.36,3169.56
17323,51.71,101.00,7.00,1106.47,0.72,1.44,1127.61,1708.35
12438,74.29,74.57,16.00,501.87,1.56,3.13,506.17,1661.76
7330,62.71,67.29,52.00,166.22,4.65,9.31,166.71,1627.93
8868,3.43,34.43,8.00,601.23,1.26,2.53,611.49,1622.71
6402,8.29,9.43,2.00,862.69,0.79,1.59,923.66,1537.63
6666,9.71,13.00,17.00,259.87,2.78,5.56,262.07,1527.82
19538,52.57,58.71,31.00,228.53,3.08,6.16,229.61,1484.91
14858,14.86,15.57,3.00,743.59,0.87,1.74,778.04,1421.92


In [54]:
# CLTV Segmentation
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,17.00,30.57,5.00,187.87,0.97,1.95,193.63,395.47,A
1,209.86,224.86,21.00,95.88,0.98,1.97,96.66,199.38,B
2,52.29,78.86,5.00,117.06,0.67,1.34,120.97,170.15,B
3,1.57,20.86,2.00,60.98,0.70,1.40,67.32,98.88,D
4,83.14,95.43,2.00,104.99,0.40,0.79,114.32,94.99,D


In [56]:
def create_cltv_df(dataframe):
    columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
               "customer_value_total_ever_online"]
    for col in columns:
        replace_with_thresholds(dataframe, col)

    dataframe = dataframe.assign(
        order_num_total=dataframe['order_num_total_ever_online'] + dataframe['order_num_total_ever_offline'],
        customer_value_total=dataframe['customer_value_total_ever_online'] + dataframe[
            'customer_value_total_ever_offline'])

    dataframe = dataframe[~((dataframe["customer_value_total"] == 0) & (dataframe["order_num_total"] == 0))]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    
    today_date = dt.datetime(2021, 6, 1)
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = (dataframe["last_order_date"] - dataframe["first_order_date"]).dt.days / 7
    cltv_df["T_weekly"] = (today_date - dataframe["first_order_date"]).dt.days / 7
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 0) & (cltv_df['recency_cltv_weekly'] > 0) & (cltv_df['T_weekly'] > 0)]

    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])
    cltv_df["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 3,
                                                                                           cltv_df['frequency'],
                                                                                           cltv_df[
                                                                                               'recency_cltv_weekly'],
                                                                                           cltv_df['T_weekly'])
    cltv_df["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6,
                                                                                           cltv_df['frequency'],
                                                                                           cltv_df[
                                                                                               'recency_cltv_weekly'],
                                                                                           cltv_df['T_weekly'])

    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                           cltv_df['monetary_cltv_avg'])

    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_df


cltv_df = create_cltv_df(df)

cltv_df.head(10)


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95,193.63,395.47,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97,96.66,199.38,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34,120.97,170.15,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40,67.32,98.88,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79,114.32,94.99,D
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.86,132.29,3.00,66.95,0.38,0.77,71.35,57.42,D
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.57,64.86,4.00,93.98,0.65,1.30,98.13,134.22,C
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.71,54.57,2.00,81.81,0.52,1.04,89.57,97.66,D
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.43,70.71,5.00,210.94,0.71,1.41,217.30,322.59,A
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.71,96.00,2.00,82.98,0.39,0.79,90.81,75.20,D
